In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import pickle
import pandas as pd
import re
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
from bs4 import Comment
import unicodedata
from spacy.cli import download
download('en_core_web_sm')
#lemmatization
nlp=spacy.load('en_core_web_sm')
tokenizer=ToktokTokenizer()
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
picklefile=open('stack_dataframe_filtered_python','rb')
stack_df=pickle.load(picklefile)
picklefile.close()

In [3]:
stack_df.head()

,index,id,id_1,title,body,owner_user_id,tags,view_count,display_name,reputation,views,body_1,parent_id,owner_user_id_1
0,66,778942,10396632,error while parsing url using python,<p>I am working on a url using python.<br>\nIf...,778942,python|urllib2|urllib,290,sam,16743,918,<p>Take a look using an http listener (or even...,10396632,1231705.0
1,124,5380,9979970,Why does python use 'else' after for and while...,<p>I understand how this construct works:</p>\...,5380,python|if-statement|for-loop|for-else,244308,Kent Boogaart,169576,19396,"<p>I read it like ""When the <code>iterable</co...",9979970,860421.0
2,217,11589463,65426142,How to write scraped data into a csv file via ...,<p>I downloaded historical stock data via the ...,11589463,python-3.x|url|python-requests|data-analysis|s...,17,Samar Pratap Singh,191,152,<p>Modified code:</p>\n<pre><code>url = &quot;...,65426142,11589463.0
3,390,4117975,50085839,How to retrieve frequency of a term in elastic...,<p>I have to find the frequency of a term in p...,4117975,python|elasticsearch,2045,Amogh Mishra,873,176,"<p>look at this page -> <a href=""https://www.e...",50085839,5219197.0
4,490,1104823,18261587,python networkx remove nodes and edges with so...,<p>In the python library networkx I would like...,1104823,python|set|conditional-statements|networkx,22847,CodeKingPlusPlus,13935,1479,"<p>Note that in Aric's for networkx==2.2, you ...",18261587,10409379.0


In [4]:
stack_df_sorted=stack_df.sort_values(by="reputation",ascending=False).reset_index()

In [5]:
stack_df_sorted.head(3)

,level_0,index,id,id_1,title,body,owner_user_id,tags,view_count,display_name,reputation,views,body_1,parent_id,owner_user_id_1
0,17050,1635517,267,395451,How to download a file over http with authoriz...,<p>I have a script that I'd like to continue u...,267,python|python-3.x|urllib,19629,Lasse V. Karlsen,362877,41691,"<p>Direct from the Py3k docs: <a href=""http://...",395451,37522.0
1,15711,1504719,267,395451,How to download a file over http with authoriz...,<p>I have a script that I'd like to continue u...,267,python|python-3.x|urllib,19629,Lasse V. Karlsen,362877,41691,<p>You can use requests.get to download file. ...,395451,9265663.0
2,14333,1374552,267,395451,How to download a file over http with authoriz...,<p>I have a script that I'd like to continue u...,267,python|python-3.x|urllib,19629,Lasse V. Karlsen,362877,41691,<p>My advice would be to maintain your 2.* bra...,395451,28380.0


In [223]:
sentence=str(stack_df_sorted.iloc[100].title)
sentence

'load dll from python'

In [224]:
sentence_nlp=nlp(sentence)

In [225]:
sentence_nlp

load dll from python

In [226]:
print('POS tagging with Spacy')
spacy_tagged=[(word,word.tag_, word.pos_) for word in sentence_nlp]
display(pd.DataFrame(spacy_tagged, columns=['Word','POS tag','Tag_type']))

POS tagging with Spacy


,Word,POS tag,Tag_type
0,load,NN,NOUN
1,dll,NN,NOUN
2,from,IN,ADP
3,python,NNP,PROPN


In [227]:
parts_of_speech=['POS','ADP','ADV','AUX','CONJ','CCONJ','DET','INTJ','NUM','PART','PUNCT','SCONJ','SYM','X','SPACE']

In [228]:
#remove html tag and escaping code
def clean_html(txt):
    soup=BeautifulSoup(txt,"lxml")
    [x.extract() for x in soup.find_all('code')]
    [x.extract() for x in soup.find_all('script')]
    [x.extract() for x in soup.find_all('style')]
    [x.extract() for x in soup.find_all('meta')]
    [x.extract() for x in soup.find_all('noscript')]
    [x.extract() for x in soup.find_all(text=lambda text:isinstance(text, Comment))]
    res=soup.get_text()
    return soup.get_text()

In [229]:
#https://github.com/ismailazdad/P5_NLP_stackoverflow/blob/main/01_stackOverflow_cleaning.ipynb
#required some help and usage of how to remove certain tokens
#the bottom functions are used as reference but are not completely used in the code
def remove_words(txt,nlp):
    doc=nlp(txt)
    list_text=[]
    for token in doc:
        if (token.pos_ not in parts_of_speech):
            list_text.append(token.text)
        join_text=" ".join(list_text)
        return join_text

def remove_accented_chars(txt):
    txt=unicodedata.normalize('NFKD', txt).encode('ascii','ignore').decode('utf-8','ignore')
    return txt

In [230]:

def full_clean(txt,nlp):
    txt=remove_words(txt,nlp)
    txt=txt.lower()
    #remove unicode characters
    txt=txt.encode('ascii','ignore').decode()
    #remove accent
    txt=remove_accented_chars(txt)
    #remove links
    txt=re.sub(r'http*\S+','',txt)
    #stop_words=stopwords.words("english")
    lemma=nltk.WordNetLemmatizer()
    txt=[lemma.lemmatize(word) for word in txt]
    return txt

#test_list=[]
#for word in full_text:
#    if (word.pos_ not in parts_of_speech):
#        test_list.append(word.text)
#    join_test=" ".join(test_list)
#print(join_test)

In [231]:
stack_df_sorted['new_body']=stack_df_sorted['body'].apply(clean_html)

In [232]:
stack_df_sorted['new_body'][0]

'I have a script that I\'d like to continue using, but it looks like I either have to find some workaround for a bug in Python 3, or downgrade back to 2.6, and thus having to downgrade other scripts as well...\nHopefully someone here have already managed to find a workaround.\nThe problem is that due to the new changes in Python 3.0 regarding bytes and strings, not all the library code is apparently tested.\nI have a script that downloades a page from a web server. This script passed a username and password as part of the url in python 2.6, but in Python 3.0, this doesn\'t work any more.\nFor instance, this:\n\nfails with this exception:\n\nApparently, base64-encoding now needs bytes in and outputs a string, and thus urlretrieve (or some code therein) which builds up a string of username:password, and tries to base64-encode this for simple authorization, fails.\nIf I instead try to use urlopen, like this:\n\nThen it fails with this exception:\n\nApparently the url parsing in this "next

In [234]:
stack_df_sorted['qa']=stack_df_sorted['title'].map(str)+' '+stack_df_sorted['new_body']

In [235]:
stack_df_sorted['qa'][0]

'How to download a file over http with authorization in python 3.0, working around bugs? I have a script that I\'d like to continue using, but it looks like I either have to find some workaround for a bug in Python 3, or downgrade back to 2.6, and thus having to downgrade other scripts as well...\nHopefully someone here have already managed to find a workaround.\nThe problem is that due to the new changes in Python 3.0 regarding bytes and strings, not all the library code is apparently tested.\nI have a script that downloades a page from a web server. This script passed a username and password as part of the url in python 2.6, but in Python 3.0, this doesn\'t work any more.\nFor instance, this:\n\nfails with this exception:\n\nApparently, base64-encoding now needs bytes in and outputs a string, and thus urlretrieve (or some code therein) which builds up a string of username:password, and tries to base64-encode this for simple authorization, fails.\nIf I instead try to use urlopen, like

In [236]:
sentence=str(stack_df_sorted.iloc[0].qa)
sentence

'How to download a file over http with authorization in python 3.0, working around bugs? I have a script that I\'d like to continue using, but it looks like I either have to find some workaround for a bug in Python 3, or downgrade back to 2.6, and thus having to downgrade other scripts as well...\nHopefully someone here have already managed to find a workaround.\nThe problem is that due to the new changes in Python 3.0 regarding bytes and strings, not all the library code is apparently tested.\nI have a script that downloades a page from a web server. This script passed a username and password as part of the url in python 2.6, but in Python 3.0, this doesn\'t work any more.\nFor instance, this:\n\nfails with this exception:\n\nApparently, base64-encoding now needs bytes in and outputs a string, and thus urlretrieve (or some code therein) which builds up a string of username:password, and tries to base64-encode this for simple authorization, fails.\nIf I instead try to use urlopen, like

In [237]:
sentence_nlp=nlp(sentence)

In [238]:
print('POS tagging with Spacy')
spacy_tagged=[(word,word.tag_, word.pos_) for word in sentence_nlp]
display(pd.DataFrame(spacy_tagged, columns=['Word','POS tag','Tag_type']))

POS tagging with Spacy


,Word,POS tag,Tag_type
0,How,WRB,ADV
1,to,TO,PART
2,download,VB,VERB
3,a,DT,DET
4,file,NN,NOUN
...,...,...,...
254,choices,NNS,NOUN
255,do,VBP,AUX
256,I,PRP,PRON
257,have,VB,VERB


In [239]:
download('en_core_web_sm')
nlp=spacy.load('en_core_web_sm')


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [241]:
txt=nlp(stack_df_sorted['qa'][0])

In [271]:
def remove1(txt,nlp):
    list_text=[]
    txt1=nlp(txt)
    for word in txt1:
        if (word.pos_ not in parts_of_speech):
            if word.text != 'I':
                list_text.append(word.text)
    return list_text
def remove2(txt,nlp):
    list_text=[]
    txt1=nlp(txt)
    for word in txt1:
        if (word.pos_ not in parts_of_speech):
            if word.text != 'I':
                list_text.append(word.text)
    join_text=" ".join(list_text)
    return join_text


In [249]:
list_text=[]
for word in txt:
    if (word.pos_ not in parts_of_speech):
        if word.text != 'I':
            list_text.append(word.text)

In [147]:
new_list=[]
for line in stack_df_sorted['qa']:
    new_list.append(remove_words(line,nlp))

In [261]:
stack_df_sorted.rename(columns={'body_1':'answer_body'},inplace=True)

In [250]:

#text=stack_df_sorted['qa']

stack_df_sorted['text']=stack_df_sorted['qa'].apply(lambda x:remove1(x,nlp))

In [262]:
stack_df_sorted

,level_0,index,id,id_1,title,body,owner_user_id,tags,view_count,display_name,reputation,views,answer_body,parent_id,owner_user_id_1,new_body,qa,text
0,17050,1635517,267,395451,How to download a file over http with authoriz...,<p>I have a script that I'd like to continue u...,267,python|python-3.x|urllib,19629,Lasse V. Karlsen,362877,41691,"<p>Direct from the Py3k docs: <a href=""http://...",395451,37522.0,I have a script that I'd like to continue usin...,How to download a file over http with authoriz...,"[download, file, http, authorization, python, ..."
1,15711,1504719,267,395451,How to download a file over http with authoriz...,<p>I have a script that I'd like to continue u...,267,python|python-3.x|urllib,19629,Lasse V. Karlsen,362877,41691,<p>You can use requests.get to download file. ...,395451,9265663.0,I have a script that I'd like to continue usin...,How to download a file over http with authoriz...,"[download, file, http, authorization, python, ..."
2,14333,1374552,267,395451,How to download a file over http with authoriz...,<p>I have a script that I'd like to continue u...,267,python|python-3.x|urllib,19629,Lasse V. Karlsen,362877,41691,<p>My advice would be to maintain your 2.* bra...,395451,28380.0,I have a script that I'd like to continue usin...,How to download a file over http with authoriz...,"[download, file, http, authorization, python, ..."
3,35281,3345402,2225682,19009932,Import arbitrary python source file. (Python 3...,<p>How can I import an arbitrary python source...,2225682,python|python-3.x|python-3.3,39887,falsetru,332328,18449,<p><strong><code>importlib</code> helper funct...,19009932,895245.0,How can I import an arbitrary python source fi...,Import arbitrary python source file. (Python 3...,"[Import, arbitrary, python, source, file, Pyth..."
4,35807,3390954,2225682,19009932,Import arbitrary python source file. (Python 3...,<p>How can I import an arbitrary python source...,2225682,python|python-3.x|python-3.3,39887,falsetru,332328,18449,"<p>Found a solution from <a href=""https://gith...",19009932,2225682.0,How can I import an arbitrary python source fi...,Import arbitrary python source file. (Python 3...,"[Import, arbitrary, python, source, file, Pyth..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126537,72967,6816559,781878,6220619,Storing data in python,<p>I need help storing data within a python pr...,781878,python,1054,mike,1,0,<p>Maybe you should look at CouchDB? You basic...,6220619,615408.0,I need help storing data within a python progr...,Storing data in python I need help storing dat...,"[Storing, data, python, need, help, storing, d..."
126538,72968,6816626,4603593,28709270,how to send an email to the list of people in ...,<p>I have a python script that searches the da...,4603593,python|email,37,Nat,1,0,<p>Look into the sendgrid api if you want to e...,28709270,4590443.0,I have a python script that searches the datab...,how to send an email to the list of people in ...,"[send, email, list, people, changing, query, p..."
126539,76854,7037340,3512307,22945394,Iterate through folders and extract shapefiles...,<p>Hi I am using sample data. In a folder call...,3512307,merge|arcpy,2933,user3512307,1,1,"<p>add:</p>\n\n<pre><code>Merge = r""F:\Extract...",22945394,3537994.0,Hi I am using sample data. In a folder called ...,Iterate through folders and extract shapefiles...,"[Iterate, folders, extract, shapefiles, geodat..."
126540,76851,7037275,9520777,49377344,Error while trying to install python in ubuntu...,"<pre class=""lang-none prettyprint-override""><c...",9520777,python|ubuntu-14.04,116,Jahnavi Lingamaneni,1,1,<p>try:</p>\n\n<pre><code>&gt;&gt; sudo apt-ge...,49377344,6200285.0,\nThe error while trying this cmd is\n,Error while trying to install python in ubuntu...,"[Error, trying, install, python, ubuntu, error..."


In [264]:
stack_df_sorted['answer']=stack_df_sorted['answer_body'].apply(clean_html)

In [266]:
stack_df_sorted['answer']

0         Direct from the Py3k docs: http://docs.python....
1         You can use requests.get to download file. Try...
2         My advice would be to maintain your 2.* branch...
3          helper function\nHere is a convenient, ready-...
4         Found a solution from  test code.\nUsing impor...
                                ...                        
126537    Maybe you should look at CouchDB? You basicall...
126538    Look into the sendgrid api if you want to e-ma...
126539              add:\n\nThen your for loop should be:\n
126540                                      try:\n\nthen:\n
126541                                                     
Name: answer, Length: 126542, dtype: object

In [274]:
def remove_links(txt,nlp):
    txt=remove2(txt,nlp)
    txt=txt.lower()
    #remove unicode characters
    txt=txt.encode('ascii','ignore').decode()
    #remove accent
    txt=remove_accented_chars(txt)
    #remove links
    txt=re.sub(r'http*\S+','',txt)
    #stop_words=stopwords.words("english")
    #lemma=nltk.WordNetLemmatizer()
    #txt=[lemma.lemmatize(word) for word in txt]
    return txt

In [281]:
stack_df_sorted['answer_unlink']=stack_df_sorted['answer'].apply(lambda x:remove_links(x, nlp))

In [282]:
stack_df_sorted['answer_unlink']

0                                          direct py3k docs
1         you use requests.get download file try sample ...
2         my advice be maintain your branch your product...
3         helper function convenient ready use helper re...
4         found solution test code using importlib.machi...
                                ...                        
126537    you look couchdb you store data json document ...
126538    look sendgrid api you want e - mail python  co...
126539                                     add your loop be
126540                                                  try
126541                                                     
Name: answer_unlink, Length: 126542, dtype: object

In [6]:
lemma=nltk.WordNetLemmatizer()
#txt=[lemma.lemmatize(word) for word in txt]

In [11]:
#txt1=nlp(stack_df_sorted['qa'])

In [ ]:
#print([lemme.Lemmatize(word) for word in ])

In [10]:
stack_df_sorted

,level_0,index,id,id_1,title,body,owner_user_id,tags,view_count,display_name,reputation,views,body_1,parent_id,owner_user_id_1
0,17050,1635517,267,395451,How to download a file over http with authoriz...,<p>I have a script that I'd like to continue u...,267,python|python-3.x|urllib,19629,Lasse V. Karlsen,362877,41691,"<p>Direct from the Py3k docs: <a href=""http://...",395451,37522.0
1,15711,1504719,267,395451,How to download a file over http with authoriz...,<p>I have a script that I'd like to continue u...,267,python|python-3.x|urllib,19629,Lasse V. Karlsen,362877,41691,<p>You can use requests.get to download file. ...,395451,9265663.0
2,14333,1374552,267,395451,How to download a file over http with authoriz...,<p>I have a script that I'd like to continue u...,267,python|python-3.x|urllib,19629,Lasse V. Karlsen,362877,41691,<p>My advice would be to maintain your 2.* bra...,395451,28380.0
3,35281,3345402,2225682,19009932,Import arbitrary python source file. (Python 3...,<p>How can I import an arbitrary python source...,2225682,python|python-3.x|python-3.3,39887,falsetru,332328,18449,<p><strong><code>importlib</code> helper funct...,19009932,895245.0
4,35807,3390954,2225682,19009932,Import arbitrary python source file. (Python 3...,<p>How can I import an arbitrary python source...,2225682,python|python-3.x|python-3.3,39887,falsetru,332328,18449,"<p>Found a solution from <a href=""https://gith...",19009932,2225682.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126537,72967,6816559,781878,6220619,Storing data in python,<p>I need help storing data within a python pr...,781878,python,1054,mike,1,0,<p>Maybe you should look at CouchDB? You basic...,6220619,615408.0
126538,72968,6816626,4603593,28709270,how to send an email to the list of people in ...,<p>I have a python script that searches the da...,4603593,python|email,37,Nat,1,0,<p>Look into the sendgrid api if you want to e...,28709270,4590443.0
126539,76854,7037340,3512307,22945394,Iterate through folders and extract shapefiles...,<p>Hi I am using sample data. In a folder call...,3512307,merge|arcpy,2933,user3512307,1,1,"<p>add:</p>\n\n<pre><code>Merge = r""F:\Extract...",22945394,3537994.0
126540,76851,7037275,9520777,49377344,Error while trying to install python in ubuntu...,"<pre class=""lang-none prettyprint-override""><c...",9520777,python|ubuntu-14.04,116,Jahnavi Lingamaneni,1,1,<p>try:</p>\n\n<pre><code>&gt;&gt; sudo apt-ge...,49377344,6200285.0


In [285]:
picklefile=open('preprocessed_stack','wb')
pickle.dump(stack_df_sorted,picklefile)
picklefile.close()

In [288]:
stack1=stack_df.drop_duplicates(subset='title')

In [289]:
stack1

,index,id,id_1,title,body,owner_user_id,tags,view_count,display_name,reputation,views,body_1,parent_id,owner_user_id_1
0,66,778942,10396632,error while parsing url using python,<p>I am working on a url using python.<br>\nIf...,778942,python|urllib2|urllib,290,sam,16743,918,<p>Take a look using an http listener (or even...,10396632,1231705.0
1,124,5380,9979970,Why does python use 'else' after for and while...,<p>I understand how this construct works:</p>\...,5380,python|if-statement|for-loop|for-else,244308,Kent Boogaart,169576,19396,"<p>I read it like ""When the <code>iterable</co...",9979970,860421.0
2,217,11589463,65426142,How to write scraped data into a csv file via ...,<p>I downloaded historical stock data via the ...,11589463,python-3.x|url|python-requests|data-analysis|s...,17,Samar Pratap Singh,191,152,<p>Modified code:</p>\n<pre><code>url = &quot;...,65426142,11589463.0
3,390,4117975,50085839,How to retrieve frequency of a term in elastic...,<p>I have to find the frequency of a term in p...,4117975,python|elasticsearch,2045,Amogh Mishra,873,176,"<p>look at this page -> <a href=""https://www.e...",50085839,5219197.0
4,490,1104823,18261587,python networkx remove nodes and edges with so...,<p>In the python library networkx I would like...,1104823,python|set|conditional-statements|networkx,22847,CodeKingPlusPlus,13935,1479,"<p>Note that in Aric's for networkx==2.2, you ...",18261587,10409379.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126537,9999649,8042791,44093597,Objects and graphing in python,<p>I am a novice at programming. The program I...,8042791,python|matplotlib|physics,43,Astupidhippo,27,33,<p>You can certainly do something like this wi...,44093597,3373942.0
126538,9999656,3082073,27240143,Installing wxpython on ubuntu 14.04,<p>When I try to open PlayOnLinux from the com...,3082073,python|ubuntu|wxpython,21365,wugology,193,33,<p>All you need is:</p>\n\n<p><code>sudo apt-g...,27240143,2673386.0
126539,9999824,3953758,25632459,python finding mac os build number,<p>How can I find my Mac OS X build number? An...,3953758,python|macos,531,andrew,438,36,"<p>To find the current working platform, I wou...",25632459,3901280.0
126540,9999927,13113012,65053520,How to use infinity in python,<p>I am working with graphs in python. I am tr...,13113012,python|algorithm|data-structures|graph|infinity,336,Luke Sharon,427,36,<p>Use <code>float(&quot;inf&quot;)</code> or ...,65053520,4541045.0


In [20]:
from langdetect import detect
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
#prepare the model
n_topics=20
tf_vectorizer=CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english',token_pattern='[a-zA-Z]+')

In [26]:
tf=tf_vectorizer.fit_transform(stack_df_sorted['title'])

In [27]:
tf_vectorizer.get_feature_names()

['able',
 'access',
 'accessing',
 'add',
 'adding',
 'address',
 'ajax',
 'algorithm',
 'allow',
 'alternative',
 'amazon',
 'anaconda',
 'analysis',
 'android',
 'apache',
 'api',
 'app',
 'append',
 'appending',
 'application',
 'apply',
 'area',
 'argparse',
 'args',
 'argument',
 'arguments',
 'array',
 'arrays',
 'ascii',
 'assign',
 'assignment',
 'async',
 'asyncio',
 'attribute',
 'attributeerror',
 'attributes',
 'audio',
 'auth',
 'authentication',
 'auto',
 'automate',
 'automatically',
 'available',
 'average',
 'avoid',
 'aws',
 'axis',
 'azure',
 'b',
 'background',
 'bad',
 'bar',
 'base',
 'based',
 'bash',
 'basic',
 'batch',
 'beautiful',
 'beautifulsoup',
 'behavior',
 'behaviour',
 'best',
 'better',
 'big',
 'bigquery',
 'bin',
 'binary',
 'biopython',
 'bit',
 'blank',
 'block',
 'body',
 'boolean',
 'boost',
 'bot',
 'boto',
 'box',
 'brackets',
 'break',
 'browser',
 'buffer',
 'build',
 'building',
 'built',
 'button',
 'byte',
 'bytes',
 'c',
 'calculate',
 '

In [28]:
#Define X and y
X=stack_df_sorted['title']
y=stack_df_sorted['tags']

In [29]:
#initialize the tfidf 
tfidf=TfidfVectorizer(analyzer="word", max_df=0.6, min_df=0.005, tokenizer=None, 
                      preprocessor=' '.join, stop_words=None, lowercase=False)

In [30]:
tfidf.fit(X)

ValueError: empty vocabulary; perhaps the documents only contain stop words